In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.optimizers import SGD
from scipy.ndimage.interpolation import shift

In [3]:
with open("history.txt",'r') as historyFile:
        lines = historyFile.readlines()
        gamehistory=[]
        for x in lines:
            x=x[1:]
            x=x[:-2]
            gamehistory.append(x)
        print(gamehistory)

        

['1, 4, 5, 0, 3, 6, 1, 8', '-1, 7, 3, 0, 2, 4, 8, 6, 5', '1, 4, 1, 8, 3, 6, 0, 5, 7, 2', '-1, 1, 3, 8, 4, 0, 5', '0, 6, 8, 4, 3, 0, 1, 5, 2, 7', '1, 7, 3, 1, 2, 0, 6, 8, 5, 4', '-1, 3, 2, 4, 6, 1, 8, 0, 5', '1, 8, 4, 7, 1, 3, 0, 6', '1, 3, 4, 1, 5, 0, 6, 2', '0, 3, 4, 2, 5, 7, 0, 1, 6, 8', '1, 1, 5, 3, 6, 8, 0, 7, 2, 4', '0, 1, 4, 2, 5, 3, 6, 7, 0, 8', '-1, 5, 8, 4, 6, 0, 7', '-1, 1, 3, 7, 0, 8, 4, 2, 5', '-1, 8, 6, 2, 5, 1, 4, 7, 3', '-1, 0, 4, 7, 3, 5, 2, 1, 6', '0, 3, 1, 5, 6, 7, 8, 2, 4, 0', '1, 8, 2, 7, 6, 1, 3, 4', '1, 2, 1, 5, 0, 7, 3, 6, 8, 4', '-1, 4, 8, 3, 0, 6, 2, 7, 1', '1, 4, 0, 2, 6, 5, 8, 3', '1, 8, 2, 6, 4, 1, 5, 0, 7, 3', '1, 8, 5, 6, 0, 4, 2, 7', '1, 4, 8, 1, 6, 2, 0, 7', '-1, 0, 6, 3, 8, 1, 5, 7, 2', '1, 1, 3, 8, 0, 5, 7, 2', '1, 8, 0, 2, 7, 3, 4, 1, 6, 5', '0, 3, 7, 4, 6, 1, 0, 8, 5, 2', '0, 5, 2, 6, 7, 1, 3, 8, 0, 4', '1, 0, 2, 5, 7, 1, 8, 6, 4, 3', '1, 1, 3, 0, 5, 2', '1, 0, 7, 2, 4, 1', '1, 3, 2, 0, 1, 5, 7, 4', '1, 7, 8, 0, 1, 4, 6, 3, 2, 5', '1, 5, 8, 1, 7, 4, 

In [4]:
inputGame=[]
output=[]

for games in gamehistory:
    if(games[0]=='-'):
        inputGame.append(-1)
        output.append(games[1:])
    
print(inputGame)
print(output)

[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]
['1, 7, 3, 0, 2, 4, 8, 6, 5', '1, 1, 3, 8, 4, 0, 5', '1, 3, 2, 4, 6, 1, 8, 0, 5', '1, 5, 8, 4, 6, 0, 7', '1, 1, 3, 7, 0, 8, 4, 2, 5', '1, 8, 6, 2, 5, 1, 4, 7, 3', '1, 0, 4, 7, 3, 5, 2, 1, 6', '1, 4, 8, 3, 0, 6, 2, 7, 1', '1, 0, 6, 3, 8, 1, 5, 7, 2', '1, 4, 6, 5, 0, 7, 2, 8, 1', '1, 7, 4, 0, 3, 2, 5', '1, 3, 8, 2, 6, 0, 1, 4, 7', '1, 3, 1, 8, 2, 7, 0', '1, 1, 4, 7, 8, 5, 0', '1, 6, 7, 2, 4, 3, 1', '1, 4, 2, 3, 1, 7, 8, 6, 0', '1, 0, 5, 8, 7, 6, 4, 2, 3', '1, 1, 3, 4, 0, 5, 6', '1, 3, 2, 5, 4, 1, 0, 7, 6', '1, 0, 5, 1, 3, 8, 4', '1, 3, 7, 5, 4, 1, 8, 2, 6', '1, 8, 0, 5, 6, 7, 4, 1, 3', '1, 8, 7, 4, 2, 3, 1, 6, 0', '1, 1, 3, 8, 4, 6, 7, 0, 5', '1, 5, 2, 1, 7, 3, 4, 8, 6', '1, 4, 8, 1, 6, 3, 5, 0, 7', '1, 4, 0, 8, 5, 7, 6, 2, 3', '1, 1, 0, 7, 6, 5, 3', '1, 6, 5, 0, 4, 8, 3', '1, 2, 1, 7, 4, 8, 5, 0, 3']


In [5]:
def createData(dataset):
        inputGame = []
        output = []
        for data in dataset:
            inputGame.append(data[1])
            output.append(data[0])

        X = np.array(inputGame).reshape((-1, len(inputGame)))
        y = to_categorical(output, num_classes=3)
        # Train and test data split
        boundary = int(0.8 * len(X))
        x_train = X[:boundary]
        x_test = X[boundary:]
        y_train = y[:boundary]
        y_test = y[boundary:]


In [6]:
model = keras.Sequential()
model.add(layers.Dense(18, input_dim=9,kernel_initializer='normal', activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(9, kernel_initializer='normal',activation='relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(1,kernel_initializer='normal'))

learning_rate = 0.001
momentum = 0.8

sgd = SGD(lr=learning_rate, momentum=momentum,nesterov=False)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 18)                180       
_________________________________________________________________
dropout (Dropout)            (None, 18)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 9)                 171       
_________________________________________________________________
dropout_1 (Dropout)          (None, 9)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 10        
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [7]:
def legal_moves_generator(current_board_state):
    """Function that returns the set of all possible legal moves and resulting board states, 
    for a given input board state and player

    Args:
    current_board_state: The current board state


    Returns:
    legal_moves_dict: A dictionary of a list of possible next coordinate-resulting board state pairs
    The resulting board state is flattened to 1 d array

    """
    current_board_state = np.array(current_board_state).reshape(3,3)
    legal_moves_dict={}
    for i in range(current_board_state.shape[0]):
        for j in range(current_board_state.shape[1]):
            if current_board_state[i,j]==0:
                board_state_copy=current_board_state.copy()
                legal_moves_dict[(i,j)]=board_state_copy.flatten()
    return legal_moves_dict

In [8]:
def move_selector(model,current_board_state):
    current_board_state = np.array(current_board_state).reshape(3,3)
    tracker={}
    legal_moves_dict=legal_moves_generator(current_board_state)
    for legal_move_coord in legal_moves_dict:
        score=model.predict(legal_moves_dict[legal_move_coord].reshape(1,9))
        tracker[legal_move_coord]=score

    selected_move=max(tracker, key=tracker.get)

    new_board_state=legal_moves_dict[selected_move]
    score=tracker[selected_move]

    return selected_move,new_board_state,score


In [9]:
import os
import random
import copy

#X's are 1, O's are -1, empty is 0
#check if 3 in a row has been made
def checkWin(board):
    win = 0
    winningLines = [[0, 1, 2],[3, 4, 5],[6, 7, 8],[0, 3, 6],[1, 4, 7],[2, 5, 8],[0, 4, 8],[2, 4, 6]]
    for x in winningLines:
        if( board[x[0]]==1 and  board[x[1]]==1 and  board[x[2]]==1):
            win = 1
        elif(board[x[0]]==-1 and  board[x[1]]==-1 and  board[x[2]]==-1):
            win = -1
    return win
#find which parts of the array are available
def findLegalMoves(board):
    legalMoves=[]
    for i in range(len(board)):
        if board[i] == 0:
            legalMoves.append(i)
    return legalMoves

#why didn't I use a dictionary
def createBoard(board):
    physicalBoard=[]
    for x in board:
        if x == -1:
            physicalBoard.append('O')
        elif x == 1:
            physicalBoard.append('X')
        elif x == 0:
            physicalBoard.append(" ")

    return physicalBoard
#display board
def printBoard(board):
    physicalBoard = createBoard(board)

    print(physicalBoard[0]+"|"+physicalBoard[1]+"|"+physicalBoard[2])
    print("-+-+-")
    print(physicalBoard[3]+"|"+physicalBoard[4]+"|"+physicalBoard[5])
    print("-+-+-")
    print(physicalBoard[6]+"|"+physicalBoard[7]+"|"+physicalBoard[8]+"\n")
def printHelpBoard():
    print("\nChoose a place to put your piece using the following guide:")
    print("0"+"|"+"1"+"|"+"2")
    print("-+-+-")
    print("3"+"|"+"4"+"|"+"5")
    print("-+-+-")
    print("6"+"|"+"7"+"|"+"8\n")



#initialize empty board
board = [0,0,0, 0,0,0 ,0,0,0]
#begin counting pieces played
piecesPlayed = 0
gameOver = False



while(piecesPlayed<9 and gameOver == False):
    #display empty board
    printBoard(board)
    #find legal moves
    legalMoves = findLegalMoves(board)
    playerInput=-1
    #State which player moves
    if(piecesPlayed%2==0):
        print("Player X's Turn")
        while (playerInput<0 or playerInput>8 or playerInput not in legalMoves):
            printHelpBoard()
            playerInput = int(input("Where would you like to place your piece?"))
            if playerInput not in legalMoves:
                print("Please choose a legal move.")
            elif( playerInput <0 or playerInput >8):
                print("Please choose a move in the game.")
            else:
                board[playerInput]=1
    else:
        print("Player O's Turn")
        print("Generating random move for O")
        playerInput,board,score=move_selector(model,board)
        playerInput=playerInput[0]*3+playerInput[1]
        board[playerInput]=-1

    piecesPlayed = piecesPlayed + 1
    win = checkWin(board)
    if (win == 1):
        print("Player X Wins")
        gameOver = True
        printBoard(board)
    elif(win == -1):
        print("Player O Wins")
        gameOver = True
        printBoard(board)
    elif(piecesPlayed == 9):
        print("Game is a draw.")
        gameOver = True
        printBoard(board)


 | | 
-+-+-
 | | 
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 0


X| | 
-+-+-
 | | 
-+-+-
 | | 

Player O's Turn
Generating random move for O
X|O| 
-+-+-
 | | 
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 0


Please choose a legal move.

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



...

Please choose a legal move.

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 5


X|O|X
-+-+-
O| |X
-+-+-
 | | 

Player O's Turn
Generating random move for O
X|O|X
-+-+-
O|O|X
-+-+-
 | | 

Player X's Turn

Choose a place to put your piece using the following guide:
0|1|2
-+-+-
3|4|5
-+-+-
6|7|8



Where would you like to place your piece? 6


X|O|X
-+-+-
O|O|X
-+-+-
X| | 

Player O's Turn
Generating random move for O
Player O Wins
X|O|X
-+-+-
O|O|X
-+-+-
X|O| 



In [12]:
def opponent_move_selector(current_board_state):
    current_board_state = np.array(current_board_state).reshape(3,3)

    legal_moves_dict=legal_moves_generator(current_board_state)
    selected_move = random.choice(list(legal_moves_dict.keys()))
    selected_move = selected_move[0]*3+selected_move[1]

    
    
    return selected_move

In [13]:
def train(model):
    """Function trains the Evaluator (model) by playing a game against an opponent 
    playing random moves, and updates the weights of the model after the game
    
    Note that the model weights are updated using SGD with a batch size of 1

    Args:
    model: The Evaluator function being trained

    Returns:
    model: The model updated using SGD
    y: The corrected scores

    """ 
    # start the game
    board=[0,0,0,0,0,0,0,0,0]
    scores_list=[]
    corrected_scores_list=[]
    new_board_states_list=[]
    piecesPlayed = 0
    #while game going
    while(piecesPlayed<9):
         #if x's turn
            if(piecesPlayed%2==0):
                selectedMove= opponent_move_selector(board)
                board[selectedMove]=1
            #else if programs turn   
            else:
                selected_move,new_board_state,score=move_selector(model,board)
                scores_list.append(score[0][0])
                new_board_states_list.append(new_board_state)
                # Make the next move
                selected_move=selected_move[0]*3+selected_move[1]
                board[selected_move]=-1                       
            piecesPlayed = piecesPlayed + 1
            win = checkWin(board)
            if (win == 1):
                print("Player X Wins")
                gameOver = True
                printBoard(board)
            elif(win == -1):
                print("Player O Wins")
                gameOver = True
                printBoard(board)
            elif(piecesPlayed == 9):
                print("Game is a draw.")
                gameOver = True
                printBoard(board)

    
    # Correct the scores, assigning 1/0/-1 to the winning/drawn/losing final board state, 
    # and assigning the other previous board states the score of their next board state
    new_board_states_list=tuple(new_board_states_list)
    new_board_states_list=np.vstack(new_board_states_list)
    if (win==-1): 
        corrected_scores_list=shift(scores_list,-1,cval=1.0)
        result="Won"
    elif (win==1):
        corrected_scores_list=shift(scores_list,-1,cval=-1.0)
        result="Lost"
    elif (win==0):
        corrected_scores_list=shift(scores_list,-1,cval=0.0)
        result="Drawn"
        
    x=new_board_states_list
    y=corrected_scores_list
    
    def unison_shuffled_copies(a, b):
        assert len(a) == len(b)
        p = np.random.permutation(len(a))
        return a[p], b[p]
    
    # shuffle x and y in unison
    x,y=unison_shuffled_copies(x,y)
    x=x.reshape(-1,9) 
    
    # update the weights of the model, one record at a time
    model.fit(x,y,epochs=1,batch_size=1,verbose=0)
    return model,y,result

ERROR! Session/line number was not unique in database. History logging moved to new session 205


In [ ]:
game_counter=1

while(game_counter<=2000):
    model,y,result=train(model)
    

Player X Wins
O|X|X
-+-+-
O|O|X
-+-+-
X|O|X

Player O Wins
O|O|O
-+-+-
 |X|X
-+-+-
 |X| 

Player X Wins
O|O|O
-+-+-
X|X|X
-+-+-
 |X| 

Player X Wins
O|O|O
-+-+-
X|X|X
-+-+-
O|X| 

Player X Wins
O|O|O
-+-+-
X|X|X
-+-+-
O|X|X

Player X Wins
O|X|O
-+-+-
O|X|O
-+-+-
X|X|X

Player X Wins
O|O|X
-+-+-
X|X|O
-+-+-
X| | 

Player X Wins
O|O|X
-+-+-
X|X|O
-+-+-
X|O| 

Player X Wins
O|O|X
-+-+-
X|X|O
-+-+-
X|O|X

Player X Wins
O|O|X
-+-+-
 | |X
-+-+-
 | |X

Player X Wins
O|O|X
-+-+-
O| |X
-+-+-
 | |X

Player X Wins
O|O|X
-+-+-
O|X|X
-+-+-
 | |X

Player X Wins
O|O|X
-+-+-
O|X|X
-+-+-
O| |X

Player X Wins
O|O|X
-+-+-
O|X|X
-+-+-
O|X|X

Player X Wins
O|X|O
-+-+-
X|O|O
-+-+-
X|X|X

Player O Wins
O|O|O
-+-+-
X| | 
-+-+-
 |X|X

Player O Wins
O|O|O
-+-+-
X|X| 
-+-+-
 |X|X

Player O Wins
O|O|O
-+-+-
X|X|O
-+-+-
 |X|X

Player X Wins
O|O|O
-+-+-
X|X|O
-+-+-
X|X|X

Player X Wins
O|O|X
-+-+-
O|X| 
-+-+-
X| |X

Player X Wins
O|O|X
-+-+-
O|X|O
-+-+-
X| |X

Player X Wins
O|O|X
-+-+-
O|X|O
-+-+-
X|X|X

Player O W

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Player X Wins
O|X|O
-+-+-
O|X| 
-+-+-
X|X| 

Traceback (most recent call last):
  File "/home/a/miniconda3/envs/RETprojects/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-14-65e6bc66abab>", line 4, in <module>
    model,y,result=train(model)
  File "<ipython-input-13-47a1e82635db>", line 29, in train
    selected_move,new_board_state,score=move_selector(model,board)
  File "<ipython-input-8-3f4170ea04bb>", line 6, in move_selector
    score=model.predict(legal_moves_dict[legal_move_coord].reshape(1,9))
  File "/home/a/miniconda3/envs/RETprojects/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training.py", line 1013, in predict
    use_multiprocessing=use_multiprocessing)
  File "/home/a/miniconda3/envs/RETprojects/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/training_v2.py", line 498, in predict
    workers=workers, use_multiprocessing=